# 1.Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics as mt
from sklearn.ensemble import RandomForestRegressor
import pickle

# 2. Funcao para carregar os datasets

In [2]:
def Load_Data_Set(df_x_train, df_y_train):
    df1 = pd.read_csv(df_x_train)
    df2 = pd.read_csv(df_y_train)
    return df1, df2

# 3. Criando e concatenando os dataframes

### 3.1 Dataset de treinamento

In [3]:
df_x_train = "../2_ensaio_regressao/1_dados_treinamento/X_training.csv"

In [4]:
df_y_train = "../2_ensaio_regressao/1_dados_treinamento/y_training.csv"

In [5]:
df1_train, df2_train = Load_Data_Set(df_x_train, df_y_train)

In [6]:
df1_train['label'] = df2_train

In [7]:
df_train = df1_train.copy()

### 3.2 Dataset de validacao

In [8]:
df_x_val = "../2_ensaio_regressao/2_dados_validacao/X_validation.csv"

In [9]:
df_y_val = "../2_ensaio_regressao/2_dados_validacao/y_val.csv"

In [10]:
df1_val, df2_val = Load_Data_Set(df_x_val, df_y_val)

In [11]:
df1_val['label'] = df2_val

In [12]:
df_val = df1_val.copy()

### 3.3 Dataset de teste

In [13]:
df_x_test = "../2_ensaio_regressao/3_dados_teste/X_test.csv"

In [14]:
df_y_test = "../2_ensaio_regressao/3_dados_teste/y_test.csv"

In [15]:
df1_test, df2_test = Load_Data_Set(df_x_test, df_y_test)

In [16]:
df1_test['label'] = df2_test

In [17]:
df_test = df1_test.copy()

# 4. Funções para treinar o modelo e avaliar as métricas dele

In [18]:
#Criando uma função para Treinar o Modelo encima dos dados de treinamento

#Fazer as previsoes encima dos proprios dados de treinamento

#Entrada: dataframe de treinamento + parametros de ajuste do modelo, a saber max_depth e n_estimators

#Saida: valores de y_train + valores de previsao feitos a partir dos dados de treinamento (y_pred_train) +
# algoritmo random forest regressor treinado

def Model_Training(df_train, max_depth, n_estimators):
    
    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label']
    
    #Preparação dos dados de treino
    x_train = df_train.loc[:, features]

    #Esse metodo ravel transforma os valores em um array
    y_train = df_train.loc[:, label].values.ravel()      
    
    #Treinamento do algoritmo Random Forest Regressor
      
    random_forest_regressor = RandomForestRegressor(max_depth = max_depth, n_estimators = n_estimators)   
  
    random_forest_regressor.fit(x_train, y_train)

    #Fazendo previsoes emcima dos dados de treinamento
    y_pred_train = random_forest_regressor.predict(x_train)
    
    return y_train, y_pred_train, random_forest_regressor
   

In [19]:
def Previsoes_Dataframe_Test(df_test, random_forest_regressor):
    
    #Entrada: dataset de teste +  algoritmo random_forest_regressor treinado
    
    #Saida: os dados de y do proprio dataset de teste (y_test) e as previsoes feitas encima dos dados de teste (y_pred_test)
    
    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label'] 
       
    #Separando os dados
    x_test = df_test.loc[:, features]

    y_test = df_test.loc[:, label].values.ravel() 
    
    #Fazendo previsões
    y_pred_test = random_forest_regressor.predict(x_test)
    
    return y_test, y_pred_test 
       

In [20]:
def Previsoes_Dataframe_Val(df_val, random_forest_regressor):
    
   #Entrada: dataset de validacao +  algoritmo random_forest_regressor treinado
    
   #Saida: os dados de y do proprio dataset de validacao (y_val) e as previsoes feitas encima dos dados de validacao
   #(y_pred_val)
    

    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label'] 
    
           
    #Separando os dados
    x_val = df_val.loc[:, features]

    y_val = df_val.loc[:, label].values.ravel() 
    
    #Fazendo previsoes
    y_pred_val = random_forest_regressor.predict(x_val)
    
    return y_val, y_pred_val
        
        

In [21]:
#Criando uma função para avaliar as metricas do modelo

#Entrada: Os proprios valores da label(y) do dataset que precisamos avaliar as metricas, 
# as previsoes feitas a partir desses dados, os parametros que utilizamos no treinamento do modelo

#Saida: Dataframe com as principais metricas do modelo

def Model_Metrics(y, y_pred,max_depth, n_estimators):
    
    #R2_Score
    r2_score = np.round(mt.r2_score(y , y_pred ),4)
        
    #Mean_Squared_Error (MSE)
    mean_squared_error = np.round(mt.mean_squared_error(y, y_pred ),4)
        
    #Root Mean_Squared Error (RMSE)
    rmse = np.round(np.sqrt(mean_squared_error),4)
        
    #Mean_Absolute_Error(MAE)
    mean_absolute_error = np.round(mt.mean_absolute_error (y,y_pred),4)
   
    #Mean Absolute Percentage Error (MAPE)
    mean_absolute_percentage_error = np.round(mt.mean_absolute_percentage_error (y,y_pred),4)
    
    #Criando um dataframe com as metricas
    
    
    parameters = 'max_depth = ' + str(max_depth) + ' ;n_estimators = ' + str(n_estimators)
    
    d = { parameters: [r2_score, mean_squared_error , rmse , mean_absolute_error ,mean_absolute_percentage_error]}  
    
    df_metrics =  pd.DataFrame(data = d, index = ['r2_score', 'mse', 'rmse', 'mae',
                                                 'mape'])      
    
        
    return df_metrics

# 5. Treinamento do modelo, validação e verificação de performance

### 5.1 Treinar o modelo com os valores de parametro default - max_depth = None, n_estimators = 100 e verificação das métricas do modelo sobre os dados de treinamento

In [22]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,None,100)
df_default_train = Model_Metrics(y_train, y_pred_train, None,100)
df_default_train

,max_depth = None ;n_estimators = 100
r2_score,0.9016
mse,47.0224
rmse,6.8573
mae,4.8835
mape,2.6602


### 5.2 Testar o algoritmo com o parametro default  nos dados de validação e verificar sua performance

In [23]:
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_default_val = Model_Metrics(y_val, y_pred_val,None,100)
df_default_val

,max_depth = None ;n_estimators = 100
r2_score,0.3339
mse,318.0481
rmse,17.8339
mae,13.0000
mape,7.1099


### 5.3 Treinar o modelo nos dados de treinamento alterando os parametros e testar a performance desse modelo sobre os dados de validação

### a) max_depth = None, n_estimators = 150

In [24]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,None, 150)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val1 = Model_Metrics(y_val,y_pred_val,None, 150)
df_val1

,max_depth = None ;n_estimators = 150
r2_score,0.3378
mse,316.1862
rmse,17.7816
mae,12.9673
mape,7.0334


### b) max_depth = None, n_estimators = 50

In [25]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,None, 50)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val2 = Model_Metrics(y_val,y_pred_val,None, 50)
df_val2

,max_depth = None ;n_estimators = 50
r2_score,0.3300
mse,319.9129
rmse,17.8861
mae,13.0724
mape,7.0163


### c) max_depth = 10, n_estimators = 100

In [26]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,10, 100)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val3 = Model_Metrics(y_val,y_pred_val,10, 150)
df_val3

,max_depth = 10 ;n_estimators = 150
r2_score,0.2120
mse,376.3012
rmse,19.3985
mae,15.2777
mape,7.8937


### d) max_depth = 5, n_estimators = 100

In [27]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,5, 100)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val4 = Model_Metrics(y_val,y_pred_val,5, 100)
df_val4

,max_depth = 5 ;n_estimators = 100
r2_score,0.0979
mse,430.7503
rmse,20.7545
mae,16.5113
mape,8.3867


### e) max_depth = 50, n_estimators = 100

In [28]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,50, 100)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val5 = Model_Metrics(y_val,y_pred_val,50, 100)
df_val5

,max_depth = 50 ;n_estimators = 100
r2_score,0.3394
mse,315.4225
rmse,17.7601
mae,12.9444
mape,6.9560


### f) max_depth = 100, n_estimators = 100

In [29]:
y_train, y_pred_train, random_forest_regressor = Model_Training(df_train,100, 100)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, random_forest_regressor)
df_val6 = Model_Metrics(y_val,y_pred_val,100, 100)
df_val6

,max_depth = 100 ;n_estimators = 100
r2_score,0.3380
mse,316.1278
rmse,17.7800
mae,12.9308
mape,7.0354


### 5.4 Verificar o parametro que obteve os melhores resultados nos dados de validação

In [30]:
df_metrics_val = pd.concat([df_default_val, df_val1, df_val2, df_val3, df_val4, df_val5, df_val6], axis = 1)
df_metrics_val

,max_depth = None ;n_estimators = 100,max_depth = None ;n_estimators = 150,max_depth = None ;n_estimators = 50,max_depth = 10 ;n_estimators = 150,max_depth = 5 ;n_estimators = 100,max_depth = 50 ;n_estimators = 100,max_depth = 100 ;n_estimators = 100
r2_score,0.3339,0.3378,0.3300,0.2120,0.0979,0.3394,0.3380
mse,318.0481,316.1862,319.9129,376.3012,430.7503,315.4225,316.1278
rmse,17.8339,17.7816,17.8861,19.3985,20.7545,17.7601,17.7800
mae,13.0000,12.9673,13.0724,15.2777,16.5113,12.9444,12.9308
mape,7.1099,7.0334,7.0163,7.8937,8.3867,6.9560,7.0354


#### Observação: Vamos manter o valor dos parametros default como referencia para o retreino do algoritmo, pois, ainda que tivemos uma pequena melhora com as mudanças, elas não foram significativas

# 6. Unificar os dados de treinamento e validação e retreinar o algoritmo, utilizando os melhores valores para os parâmetros

### 6.1 Dividir novamente os datasets de treinamento e validação

In [31]:
features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
label = ['label'] 

#Separação dos dados de treino
x_train = df_train.loc[:, features]

#Esse metodo ravel transforma os valores em um array
y_train = df_train.loc[:, label].values.ravel()      

#Separação dos dados de validação
x_val = df_val.loc[:, features]

y_val = df_val.loc[:, label].values.ravel() 

#Separação dos dados de teste
x_test = df_test.loc[:, features]

y_test =  df_test.loc[:, label].values.ravel() 

#Unificando as features e a label dos dados de treino com as dos dados de validação

x_final = np.concatenate( (x_train, x_val) )

y_final = np.concatenate((y_train, y_val))

### 6.2 Treinar o modelo novamente com o melhor parametro obtido e fazer previsões encima dos dados de teste

In [32]:
model_last = RandomForestRegressor(max_depth = None, n_estimators = 100)    
model_last = model_last.fit(x_final, y_final)

In [33]:
y_pred_test = model_last.predict(x_test)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


### 6.3 Verificando as métricas do modelo com os dados de teste

In [34]:
df_metrics_test = Model_Metrics(y_test, y_pred_test, None, 100)
df_metrics_test

,max_depth = None ;n_estimators = 100
r2_score,0.4001
mse,292.1159
rmse,17.0914
mae,12.2526
mape,6.3071


# 7. Resguardando os objetos com as métricas de maior performance

### 7.1 Alterando o nome das colunas dos dataframes

#### a) dataframe teste com melhores métricas

In [35]:
df_metrics_test = df_metrics_test.rename(columns ={"max_depth = None ;n_estimators = 100": "Random Forest Regressor"})
df_metrics_test

,Random Forest Regressor
r2_score,0.4001
mse,292.1159
rmse,17.0914
mae,12.2526
mape,6.3071


#### b) dataframe validação com melhores métricas

In [36]:
df_default_val = df_default_val.rename(columns = {"max_depth = None ;n_estimators = 100": "Random Forest Regressor"})
df_default_val

,Random Forest Regressor
r2_score,0.3339
mse,318.0481
rmse,17.8339
mae,13.0000
mape,7.1099


#### c) dataframe treinamento com melhores métricas

In [37]:
df_default_train = df_default_train.rename(columns ={"max_depth = None ;n_estimators = 100": "Random Forest Regressor"})
df_default_train

,Random Forest Regressor
r2_score,0.9016
mse,47.0224
rmse,6.8573
mae,4.8835
mape,2.6602


# 8. Salvar os objetos com as melhores metricas em um arquivo pickle

### a) dataframe com métricas dos dados de treinamento

In [38]:
with open('arquivo_rf_regressor_train.pkl', 'wb') as arquivo_rf_regressor_train:
    pickle.dump(df_default_train,arquivo_rf_regressor_train)  

In [39]:
arquivo_rf_regressor_train.close()

### b) dataframe com métricas dos dados de validação

In [40]:
with open('arquivo_rf_regressor_val.pkl', 'wb') as arquivo_rf_regressor_val:
    pickle.dump(df_default_val,arquivo_rf_regressor_val) 

In [41]:
arquivo_rf_regressor_val.close()

### c) dataframe com métricas dos dados de teste

In [42]:
with open('arquivo_rf_regressor_teste.pkl', 'wb') as arquivo_rf_regressor_teste:
    pickle.dump(df_metrics_test,arquivo_rf_regressor_teste) 

In [43]:
arquivo_rf_regressor_teste.close()